In [1]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import time
from IPython.display import display

# Project libraries
import constants

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"
# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

## TODO
debug = True # Print the data frames
completion_flag = False # completion flag

In [2]:

# liability_amount = 100

In [3]:
#######################################
# Filter out only greyhoud races
#######################################
gh_racing_id = constants.GH_RACING_ID #Greyhound racing ID
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

if debug:
    # Create a DataFrame with all the events by iterating over each event object
    gb_gh_events_df = pd.DataFrame({
        'Event Name': [event_object.event.name for event_object in gb_gh_events],
        'Event ID': [event_object.event.id for event_object in gb_gh_events],
        'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
        'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
        'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
        'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
        'Market Count': [event_object.market_count for event_object in gb_gh_events]
    })

    display(gb_gh_events_df)

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Central Park (F/C) 3rd Jul,31567489,Central Park,GB,Europe/London,2022-07-03 17:09:00,5
1,Henlow (F/C) 3rd Jul,31567505,Henlow,GB,Europe/London,2022-07-03 17:17:00,5
2,Doncaster (F/C) 3rd Jul,31567440,Doncaster,GB,Europe/London,2022-07-03 09:46:00,1
3,Kinsley (F/C) 3rd Jul,31567458,Kinsley,GB,Europe/London,2022-07-03 13:04:00,12
4,Pelaw Grange (F/C) 3rd Jul,31567428,Pelaw Grange,GB,Europe/London,2022-07-03 10:43:00,3
5,Towcester (F/C) 3rd Jul,31567467,Towcester,GB,Europe/London,2022-07-03 12:57:00,14
6,Swindon (F/C) 3rd Jul,31567482,Swindon,GB,Europe/London,2022-07-03 17:19:00,5
7,Sunderland (F/C) 3rd Jul,31567420,Sunderland,GB,Europe/London,2022-07-03 10:06:00,2
8,Harlow (F/C) 3rd Jul,31567452,Harlow,GB,Europe/London,2022-07-03 09:53:00,1


In [4]:
#######################################
#TODO: extract a list of IDs for the forecast markets
#######################################
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31567489',
  '31567505',
  '31567440',
  '31567458',
  '31567428',
  '31567467',
  '31567482',
  '31567420',
  '31567452'],
 ['Central Park (F/C) 3rd Jul',
  'Henlow (F/C) 3rd Jul',
  'Doncaster (F/C) 3rd Jul',
  'Kinsley (F/C) 3rd Jul',
  'Pelaw Grange (F/C) 3rd Jul',
  'Towcester (F/C) 3rd Jul',
  'Swindon (F/C) 3rd Jul',
  'Sunderland (F/C) 3rd Jul',
  'Harlow (F/C) 3rd Jul'])

In [5]:
#######################################
## Filter out the next 5 upcoming games
#######################################
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

## show the dataframes if set
if debug:
    market_types_venueOfTheDay_df = pd.DataFrame({
        'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
        'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
        'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
        'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
        'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
        
    })

    display(market_types_venueOfTheDay_df)

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200715893,2022-07-03 12:24:00,0.0000,Harlow
1,Forecast,1.200715820,2022-07-03 12:28:00,0.0000,Pelaw Grange
2,Forecast,1.200715858,2022-07-03 12:33:00,0.0000,Doncaster
3,Forecast,1.200715791,2022-07-03 12:36:00,46.0616,Sunderland
4,Forecast,1.200715894,2022-07-03 12:39:00,0.0000,Harlow


In [6]:
#######################################
## Check the start time of the next game
## and sleep until 15 seconds before the game
#######################################
# from utils import get_next_market

# time_gap, myRaceID, myRaceVenue = get_next_market(market_catalogues)
# # assert myRaceID == 0, "ERROR: myRaceID = 0" #TODO: remove in production

# if(time_gap > datetime.timedelta(seconds=constants.PREBET_DELAY)):
#     time_to_sleep = (time_gap - datetime.timedelta(seconds=constants.PREBET_DELAY)).seconds
#     print("Sleeping for " + str(time_to_sleep) + " seconds")
#     # time.sleep(time_to_sleep)
#     print("Sleeping done")
# else:
#     print("Don't need to sleep")



In [7]:
myRaceID = '1.200715858'
myRaceVenue = 'doncaster'
print("my race id = ", myRaceID, "my race venue = ", myRaceVenue)
#######################################
# Get a list of lay options
#######################################
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]
runners_df = process_runner_books(market_book.runners)
if debug:
    display(runners_df)

#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)
    
# if debug:
print(lay_options_ids)

#######################################
# Get odds from NEDS
#######################################
from utils import choose_lay_option_neds

lay_selection_index = choose_lay_option_neds(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    # assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    # assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print("lay_selection_id = " + str(lay_selection_id))
# fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']
fav_price = 5



my race id =  1.200715858 my race venue =  doncaster


,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,23.0,48.31,60.0,25.42,None,0.0,ACTIVE,None,None
1,41419586,19.0,59.05,36.0,34.03,None,0.0,ACTIVE,None,None
2,41419587,23.0,48.31,42.0,32.76,None,0.0,ACTIVE,None,None
3,41419588,27.0,40.89,60.0,18.02,None,0.0,ACTIVE,None,None
4,41419589,22.0,50.61,55.0,24.20,None,0.0,ACTIVE,None,None
5,41419592,28.0,42.89,80.0,26.57,None,0.0,ACTIVE,None,None
6,41419593,32.0,8.86,70.0,20.73,None,0.0,ACTIVE,None,None
7,41419594,32.0,34.28,70.0,15.41,None,0.0,ACTIVE,None,None
8,41419595,46.0,23.62,110.0,9.74,None,0.0,ACTIVE,None,None
9,41419596,44.0,24.71,100.0,16.72,None,0.0,ACTIVE,None,None


[41419585, 41419586, 41419587, 41419588, 41419589, 41419592, 41419593, 41419594, 41419595, 41419596, 41419599, 41419600, 41419601, 41419602, 41419603, 41419606, 41419607, 41419608, 41419609, 41419610, 41419613, 41419614, 41419615, 41419616, 41419617, 41419620, 41419621, 41419622, 41419623, 41419624]
['DONCASTER BAGS', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '450m', '450Mtrs (B6)', '6m', 'RACE FIELD', 'COMMENTS', '$ TRACKER', 'FLUCTUATIONS', 'WIN / PLACE SAME RACE MULTI BLENDED MBO', 'LIVE VIDEO', 'Doncaster Bags', 'FIXEDSTARTING PRICE', 'RUNNERS']
end of part1
['RUNNERS', 'WIN', 'PLC', '1343', '1. Blakes Lace (1)', 'T: J Simpson', '4.00', '2.05', '5554', '2. Lees Little Girl (2)', 'T: L J Macmanus', '7.00', '3.10', '2131', '3. Hulla Head (3)', 'T: S Atkinson', '3.50', '1.90', '3634', '4. Swift Radha (4)', 'T: G Vincent', '3.90', '2.05', '6315', '5. Swift Emphasis (5)', 'T: D L Cross', '5.50', '2.60', '4163', '6. Eden Lane (6)', 'T: M Haythorne', '4.50', '2.25', '

In [8]:
#######################################
# Choose the liability amount
# random from a list of 10 options
#######################################
import numpy as np
import constants
liability_options = []

if(len(liability_options) == 0):
    liability_options = [3,3] #not sure why this can't be moved into constants.py
print("liability_options [BEFORE] = ", liability_options, ", LENGTH =", len(liability_options))

[liability_amount] = np.random.choice(liability_options, size=1)
print("Chosen Liability amount = $", liability_amount)

liability_options.remove(liability_amount)
print("liability_options [AFTER] = ", liability_options, ", LENGTH =", len(liability_options))

liability_options [BEFORE] =  [3, 3] , LENGTH = 2
Chosen Liability amount = $ 3
liability_options [AFTER] =  [3] , LENGTH = 1


In [9]:
#######################################
# Choose order type (limit order or market_on_close order - choose the latter)
#######################################
order_filter = filters.limit_order(
    price=str(fav_price),
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=str(liability_amount) 
)
print(order_filter)

#######################################
# Create a place instruction filter
#######################################
instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    side="LAY",
    order_type = "LIMIT", # fixed price order
    limit_order=order_filter
)
print(instructions_filter)


{'price': '5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '3'}
{'orderType': 'LIMIT', 'selectionId': '41419601', 'side': 'LAY', 'limitOrder': {'price': '5', 'persistenceType': 'LAPSE', 'betTargetType': 'PAYOUT', 'betTargetSize': '3'}}


In [10]:
#######################################
# Place the order
#######################################
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [11]:
# #######################################
# # List the cleared orders
# #######################################
# import utils
# myRaceID = 1.200450524 # lost raceID

# # time.sleep(5)
# # current_orders = trading.betting.list_current_orders(
# #     customer_strategy_refs=['Naing_maker']).__dict__
# # data = pd.DataFrame(current_orders['currentOrders']).head()
# # print(current_orders)

# #######################################
# # Check if the last bet has settled and note the result
# #######################################
# settled_flag = False
# while settled_flag == False:
#     cleared_orders = trading.betting.list_cleared_orders(
#         bet_status="SETTLED",
#         market_ids=[myRaceID])

#     if len(cleared_orders._data['clearedOrders']) != 0:
#         settled_flag = True

#         # Create a DataFrame from the orders
#         betResult = pd.DataFrame(cleared_orders._data['clearedOrders'])
#         print(betResult)

#         betOutcome = betResult['betOutcome'][0]
#         profit = 0
#         for i in betResult['profit']:
#             profit = profit + float(i)

#         ## Record the results into a csv file
#         utils.write_to_file(constants.F_NAME, myRaceID, betOutcome, profit)

#         completion_flag = True # we are ready for next game

#     else:
#         print("Sleep 60 seconds before checking again if market is settled")
#         time.sleep(5) # TODO:Check again in 60 seconds
        
    

